# 1.验证模型结构

In [ ]:
import torch
import numpy as np
# 检测GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# 测试用输入
batch =  []
batch_size = 32
feature_channels = 34
feature_height = 4
feature_width = 34

action_space = 49

seq_lens = []
for i in range(batch_size):
    seq_lens.append(np.random.randint(1, 129))

for seq_len in seq_lens:
    
    action_list = [i for i in range(seq_len)]
    self_action_mask = np.random.randint(0, 2, (seq_len))
    num_self_actions = np.sum(self_action_mask)
    legal_action_mask = np.random.randint(0, 2, (num_self_actions, action_space))

    info = {
        'scores': np.random.rand(num_self_actions, 4),
        'oya': np.random.randint(0, 4, (num_self_actions,)),
        'honba_riichi_sticks': np.random.randint(0, 8, (num_self_actions, 2)),
    }

    tile_features = np.random.randint(0, 2, (num_self_actions, feature_channels, feature_height, feature_width))
    batch.append({
        'action_list': action_list,
        'info': info,
        'tile_features': tile_features,
        'reward': 1000,
        'self_action_mask': self_action_mask,
        'legal_action_mask': legal_action_mask
    })

for i, sample in enumerate(batch):
    action_list = sample['action_list']
    self_action_mask = sample['self_action_mask']
    legal_action_mask = sample['legal_action_mask']
    info = sample['info']
    tile_features = sample['tile_features']
    print("action_list len:", len(action_list))
    print("scores shape:", info['scores'].shape)
    print("self_action_mask shape:", self_action_mask.shape)
    print("legal_action_mask shape:", legal_action_mask.shape)
    print("oya shape:", info['oya'].shape)
    print("honba_riichi_sticks shape:", info['honba_riichi_sticks'].shape)
    print("tile_features shape:", tile_features.shape)
    print("-----------------------------------------------------")
    if i == 3:
        break

In [ ]:
from base_module import myCollator
# 验证collator
collator = myCollator()
data = collator(batch)

In [ ]:
print("action_list shape:", data['action_list'].shape)
print("mask shape", data['mask'].shape)
print(data['mask'][:1])
print("self_action_mask shape:", data['self_action_mask'].shape)
# print("self_action_mask:", data['self_action_mask'][:1])
num_self_actions = data['self_action_mask'].sum().item()
print("num_self_actions", num_self_actions)
print("legal_action_mask shape:", data['legal_action_mask'].shape)
print("---------------------------------------------")
print("scores shape:", data['info']['scores'].shape)
print("oya shape:", data['info']['oya'].shape)
print("honba_riichi_sticks shape:", data['info']['honba_riichi_sticks'].shape)
print("---------------------------------------------")
print("tile_features shape:", data['tile_features'].shape)
print("Q shape:", data['Q'].shape)
print("Q:", data['Q'][:50])


In [ ]:
from base_module import *
import torch
import torch.nn as nn
import numpy as np
from transformers import GPT2Model, GPT2Config



# 创建模型
config = GPT2Config(n_embd=512, n_layer=8, n_head=8, n_positions=128)
model = Policy_Network(config).to(device)


In [ ]:
model.eval()
output = model.forward(**data)
print("action_logits shape:", output['action_logits'].shape)
# print(output['action_logits'][0])
print("action_probs shape:", output['action_probs'].shape)
# print(output['action_probs'][0])
print("loss :", output['loss'])

# 2.验证麻将环境

In [1]:
from pymahjong import *
from pymahjong import MahjongPyWrapper as pm
from pymahjong.myEnv_pymahjong import myMahjongEnv

env = myMahjongEnv()
env.reset()

In [2]:
while not env.is_over():
    curr_pid = env.get_curr_player_id()
    valid_actions = env.get_valid_actions()
    action = np.random.choice(valid_actions)
    env.step(player_id=curr_pid, action=action)
print(env.get_payoffs())

[0. 0. 0. 0.]


#### 展示当前phase和合法动作列表

In [ ]:
phase, aviable_action = env._proceed()
print("phase:", phase)
for idx, action in enumerate(aviable_action):
    print(idx, action.to_string())

valid_action = env.get_valid_actions()
print("valid_action:", valid_action)

#### 执行动作

In [5]:
env.step(0, 1)

In [3]:
for i in range(4):
    print("player", i, "hand:", env._get_hand_tiles(i))

player 0 hand: ['2m', '5m', '8m', '5s', '5s', '5z', '9s']
player 1 hand: ['3m', '6m', '9m', '4s']
player 2 hand: ['1m', '5m', '4s', '2z', '3z', '3z', '4z']
player 3 hand: ['8p', '7s', '7s', '9s']


In [3]:
obs = env.get_observation(0)
print("tiles_features shape:", obs['tiles_features'].shape)
print("oya shape:", obs['info']['oya'].shape)
print("riichi_sticks shape:", obs['info']['riichi_sticks'].shape)
print("action_list shape:", obs['action_list'].shape)
print("action_list:", obs['action_list'])
print("self_action_mask shape:", obs['self_action_mask'].shape)
print("self_action_mask:", obs['self_action_mask'])
print("sum self_action_mask:", obs['self_action_mask'].sum())
print("padding_mask shape:", obs['padding_mask'].shape)
print("padding_mask:", obs['padding_mask'])

tiles_features shape: (22, 35, 4, 9)
oya shape: (22,)
riichi_sticks shape: (22,)
action_list shape: (128,)
action_list: [141  32  58 104 142  17  74 114 177 154  32  70 178 170  26  57 110 172
  29  63  96 170   2  71 127 163   5  47 106 163  18  81 131 122 178 160
  84  71 127 154  16  52 129 126 143  21  82  56 124  37  31  79 106 149
  28  58 107 153  16  68 103 142   3  54  37   3  59 100 143  27  60  94
 145  22  67 125 142  17  58 119 156   0  61 125 171   6  74 120  84  61
 119   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
self_action_mask shape: (128,)
self_action_mask: [0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1
 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
sum self_action_mask: 22
padding_mask shape: (128

In [ ]:
for i in range(4):
    print("player", i, "fuuros:", env._get_fuuros(i))
    print("points:", env._get_points(i))

In [ ]:
# print("dora indicator:", env._get_dora_indicators())
for i in range(10):
    print("game record:", env._get_action_record()[i])
    print("player3", env._change_action_record_aspects(3)[i])

#### 查看phase和状态

In [ ]:
table.players[1].get_river().to_string()

In [ ]:
phase = table.get_phase()
last_action = table.last_action
honba = table.honba
oya = table.oya
riichi_sticks = table.riichibo
dora = table.get_dora()
yama = table.yama

print("phase:", phase)
print("last_action:", last_action)
print("honba:", honba)
print("oya:", oya)
print("riichi_sticks:", riichi_sticks)
print("dora:", dora[0])
print("yama left:", table.get_remain_tile())
for tile in yama:
    print("id:", tile.id, "tile:", tile.tile, "is_red:", tile.red_dora)


